In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from datetime import datetime
import re

In [3]:
conf=SparkConf().setAppName("Number4").setMaster("local[*]")
sc=SparkContext(conf=conf)

In [23]:
read_line_log=sc.textFile("/home/jovyan/3_5_Independent_work/logfiles.log")
print(read_line_log.take(5))

['28.225.186.85 - - [01/Jan/2025:00:00:00 +0530] "POST /docs/api HTTP/1.0" 200 4992 "https://simmons.info/tagterms.html" "Mozilla/5.0 (iPhone; CPU iPhone OS 17_5_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.5 Mobile/15E148 Safari/604.1" 1425', '28.225.186.85 - - [01/Jan/2025:00:26:03 +0530] "DELETE /auth/register HTTP/1.0" 200 5009 "https://robles.com/tags/wp-contentsearch.asp" "Mozilla/5.0 (Linux; Android 14; Pixel 8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Mobile Safari/537.36" 3557', '82.67.67.141 - - [01/Jan/2025:00:52:07 +0530] "GET /blog/latest HTTP/1.0" 401 4951 "https://simmons.info/tagterms.html" "Mozilla/5.0 (iPhone; CPU iPhone OS 17_5_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.5 Mobile/15E148 Safari/604.1" 1182', '115.106.252.49 - - [01/Jan/2025:01:18:11 +0530] "PUT /api/v1/products HTTP/1.0" 200 4921 "https://simmons.info/tagterms.html" "Mozilla/5.0 (iPhone; CPU iPhone OS 17_5_1 like Mac OS X) AppleWebKit/605

In [34]:
def parse_log(log_line):
    ip_pattern = r'(\d+\.\d+\.\d+\.\d+)'
    method_pattern = r'(".*?")'
    que_pattern = r'"\s+(\d{3})\s+'
    bit_pattern = r'"\s+\d{3}\s+(\d+)'
    ip_match = re.search(ip_pattern, log_line).group(1)
    method_match = re.search(method_pattern, log_line).group(1)
    status_match = int(re.search(que_pattern, log_line).group(1))
    size_match = int(re.search(bit_pattern, log_line).group(1))
    return(ip_match, method_match, status_match, size_match)

In [46]:
clear_log=(read_line_log.map(parse_log)
)

In [47]:
total_size=clear_log.map(lambda x: x[3]).reduce(lambda x, y: x + y)

In [66]:
#total_query=clear_log.map(lambda x: (x[2], 1)).reduceByKey(lambda x, y: x + y)
total_query=clear_log.map(lambda x: x[2]).countByValue()

print(sorted(total_query.items()))

[(200, 3850), (301, 780), (302, 773), (400, 734), (401, 781), (403, 735), (404, 820), (500, 797), (502, 730)]


In [87]:
top_5_endpoint=clear_log.map(lambda x: x[1].split()).map(lambda x: x[1]).countByValue()
top_5_endpoint=sorted(top_5_endpoint.items(), key=lambda x: x[1], reverse=True)[:5]

top_5_type=clear_log.map(lambda x: x[1].split()).map(lambda x: x[0]).countByValue()
top_5_type=sorted(top_5_type.items(), key=lambda x: x[1], reverse=True)
print(top_5_endpoint)
print(top_5_type)

[('/search?q=spark', 952), ('/docs/api', 949), ('/admin/dashboard', 935), ('/auth/register', 929), ('/', 913)]
[('"DELETE', 1711), ('"GET', 1677), ('"HEAD', 1669), ('"OPTIONS', 1662), ('"POST', 1641), ('"PUT', 1640)]


In [45]:
print(f"Общее количество запросов: {clear_log.count()}")
print(f"Средний размер ответа сервера в (байтах): {total_size // clear_log.count()}")
print(f"Количество уникальных IP-адресов: {clear_log.distinct().count()}")

Общее количество запросов: 10000
Средний размер ответа сервера в (байтах): 4999
Количество уникальных IP-адресов: 10000


In [72]:
for key, value in sorted(total_query.items()):
    print(f"{key}: {value}")

print(f"Доля успешных запросов: {total_query[200]/clear_log.count() * 100}%")

200: 3850
301: 780
302: 773
400: 734
401: 781
403: 735
404: 820
500: 797
502: 730
Доля успешных запросов: 38.5%


In [91]:
print("Топ-5 самых часто запрашиваемых реурсов")
for key, value in top_5_endpoint:
    print(f"{key}: {value} запросов")

print("Распределение по типам запросов")
for key, value in top_5_type:
    print(f"{key[1::]}: {value} запросов")

Топ-5 самых часто запрашиваемых реурсов
/search?q=spark: 952 запросов
/docs/api: 949 запросов
/admin/dashboard: 935 запросов
/auth/register: 929 запросов
/: 913 запросов
Распределение по типам запросов
DELETE: 1711 запросов
GET: 1677 запросов
HEAD: 1669 запросов
OPTIONS: 1662 запросов
POST: 1641 запросов
PUT: 1640 запросов
